<a href="https://colab.research.google.com/github/andresawa/tsa/blob/main/ASTO_E4_AP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Universidade Federal do Rio Grande  
Instituto de Oceanografia  
Programa de Pós-graduação em Oceanologia  
**Disciplina**: Análises de Séries Temporais em Oceanografia – 2023  
**Estudante**: Andrés Eloy Piñango Jauregui (153423)  

# Lista de exercícios 4: Séries de Fourier   
****
**Atividades**:  
Neste exercício, quatro séries temporais de temperatura da superfície do mar (TSM) e da concentração de clorifila-a (CHL), estimadas por sensores a bordo de satélites, são fornecidas para análises harmônicas (séries de Fourier). As séries foram obtidas dos locais mostrados na figura abaixo:  
  
![Mapa](https://raw.githubusercontent.com/andresawa/tsa/main/Data/map.png)  
  
As imagens de TSM são oriundas do sensor MODIS, a bordo do satélite AQUA da NASA. As imagens de CHL são oriundas dos sensores SeaWiFS (até 2002) e MODIS (após 2002). As imagens de TSM e CHL são composições semanais, com resolução de 9 km. A NASA, ao construir as imagens semanais, considera que elas são formadas por 8 dias consecutivos. Portanto, as imagens são formadas pelas médias das imagens deste 8 dias consecutivos. No final, 46 imagens semanais compõem o ano.  
  
As imagens foram obtidas de http://oceancolor.gsfc.nasa.gov/  

****
### Parte I – Série de Fourier
#### 1.	Visualização dos dados (series_chl_ssh_v2.mat)
**a) Antes de iniciar as atividades, verifique o tamanho das séries. Perceba que são diferentes. Use apenas 18 anos de dados da série de CHL e 14 anos da série de TSM. Ou seja, use apenas 18 * 46 (=828) dados de CHL e 14 * 46 (=644) dados de TSM. Observe que a frequência de amostragem (fs) é uma imagem por semana**

In [ ]:
### ------------------------------------------------------------------------------------
### Load the libraries to be used in the analysis
### ------------------------------------------------------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import io

In [ ]:
### ------------------------------------------------------------------------------------
### Download the mat file
### ------------------------------------------------------------------------------------
!wget https://github.com/andresawa/tsa/raw/main/Data/series_chl_sst_v2.mat

In [ ]:
### ------------------------------------------------------------------------------------
### Load the mat file
### ------------------------------------------------------------------------------------
data = io.loadmat("./series_chl_sst_v2.mat")

In [ ]:
### ------------------------------------------------------------------------------------
### Transform the data into a table
### ------------------------------------------------------------------------------------
chla = pd.DataFrame(
    {
        "P1": data["chl_P1"].flatten(),
        "P2": data["chl_P2"].flatten(),
        "P3": data["chl_P3"].flatten(),
        "P4": data["chl_P4"].flatten(),
    }
)

sst = pd.DataFrame(
    {
        "P1": data["sst_P1"].flatten(),
        "P2": data["sst_P2"].flatten(),
        "P3": data["sst_P3"].flatten(),
        "P4": data["sst_P4"].flatten(),
    }
)

In [ ]:
### ------------------------------------------------------------------------------------
### Transform the time values to actual time data
### ------------------------------------------------------------------------------------
a = np.datetime64("0000-01-01")  # Epoch used by matlab
b = np.datetime64("1970-01-01")  # Unix epoch, default in python
diff = b - a
chla["time"] = pd.to_datetime(
    np.round(data["chl_time"].flatten()) - diff.astype(int), unit="D"
)
sst["time"] = pd.to_datetime(
    np.round(data["sst_time"].flatten()) - diff.astype(int), unit="D"
)

In [ ]:
### ------------------------------------------------------------------------------------
### Set the time index
### ------------------------------------------------------------------------------------
chla = chla.set_index("time")
sst = sst.set_index("time")

In [ ]:
### ------------------------------------------------------------------------------------
### Subset the data (18 years for chla, 14 for sst)
### ------------------------------------------------------------------------------------
chla_subset = chla.query("19980701 <= time <= 20160731")
sst_subset = sst.query("20021001 <= time <= 20161031")

#### 2. Eliminação de tendências e cálculo das anomalias
**a) Escolha as séries de TSM e CHL em um determinado local (lon, lat) para as análises**  
  
Ponto escolhido: `P4`  
  
**b) Plote ambas as séries na mesma figura (subplot.m)**

In [ ]:
### ------------------------------------------------------------------------------------
### Plot the series
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(10, 5), dpi=300)
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212, sharex=ax1)
### Subplot 1: Chla
chla_subset.P4.plot(ax=ax1, color="green")
ax1.grid(linestyle="dashed", alpha=0.3)
ax1.set_title("Chlorophyll-a Concentration - P4 (37°S, 50°W)", loc="left")
ax1.set_ylabel("Chla (mg $m^{-1}$)")
### Subplot 2: SST
sst_subset.P4.plot(ax=ax2, color="red")
ax2.grid(linestyle="dashed", alpha=0.3)
ax2.set_title("Sea Surface Temperature - P4 (37°S, 50°W)", loc="left")
ax2.set_ylabel("SST (°C)")
### Final touch
fig.tight_layout()
plt.show()

**c) Analisando visualmente as séries, é possível perceber alguma tendência nas séries? É possível perceber visualmente alguma relação entre TSM e CHL? Se sim, explique a razão da dependência**  

Nem SST nem clorofila parecem ter uma tendência no ponto escolhido (P4). Aparentemente, os maiores valores de clorofila encontram-se relacionados com os menores valores de temperatura. Essa relação pode ser devido a dois processos diferentes:

* Durante o inverno, a coluna de água tem uma menor estabilidade e ocorre uma maior mistura, o que traz nutrientes para a zona eufótica e aumenta a produtividade primária.  

* Ocorre um processo de ressurgência costeira, o que traz águas frias e nutrientes ao mesmo tempo, o que explica a correlação das duas variáveis.

**d) No caso da série de CHL, aplique a função logarítmo log10(CHL) para converter os valores de CHL, e refaça o item (b)**

In [ ]:
### ------------------------------------------------------------------------------------
### Plot the series
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(10, 5), dpi=300)
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212, sharex=ax1)
### Subplot 1: Chla
np.log10(chla_subset.P4).plot(ax=ax1, color="green")
ax1.grid(linestyle="dashed", alpha=0.3)
ax1.set_title("Chlorophyll-a Concentration - P4 (37°S, 50°W)", loc="left")
ax1.set_ylabel("log$_{10}$[Chla] (mg $m^{-1}$)")
### Subplot 2: SST
sst_subset.P4.plot(ax=ax2, color="red")
ax2.grid(linestyle="dashed", alpha=0.3)
ax2.set_title("Sea Surface Temperature - P4 (37°S, 50°W)", loc="left")
ax2.set_ylabel("SST (°C)")
### Final touch
fig.tight_layout()
plt.show()

**e) Remova a tendência das séries, se existir (detrend.m)**

In [ ]:
### ------------------------------------------------------------------------------------
### Define a function to detrend the data
### ------------------------------------------------------------------------------------
def linear_detrend(series):
    new_series = series.reset_index(drop=True)
    clean_series = new_series.dropna()
    regression = np.polynomial.Polynomial.fit(
        clean_series.index.values, clean_series.values, deg=1
    )
    coef = regression.convert().coef
    detrended_series = series - ((coef[1] * new_series.index.values) + coef[0])
    return detrended_series

In [ ]:
### ------------------------------------------------------------------------------------
### Detrend the data
### ------------------------------------------------------------------------------------
chla_p4 = linear_detrend(np.log10(chla.P4.squeeze()))
sst_p4 = linear_detrend(sst.P4.squeeze())

**f) Plote novamente as séries de TSM e CHL. Se os itens acima foram executados corretamente, você estará vendo agora as anomalias da TSM e do log10(CHL)**

In [ ]:
### ------------------------------------------------------------------------------------
### Plot the detrended series
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(10, 5), dpi=300)
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212, sharex=ax1)
### Subplot 1: Chla
chla_p4.plot(ax=ax1, color="green")
ax1.grid(linestyle="dashed", alpha=0.3)
ax1.set_title("Chlorophyll-a Concentration anomaly - P4 (37°S, 50°W)", loc="left")
ax1.set_ylabel("log$_{10}$[Chla] (mg $m^{-1}$)")
### Subplot 2: SST
sst_p4.plot(ax=ax2, color="red")
ax2.grid(linestyle="dashed", alpha=0.3)
ax2.set_title("Sea Surface Temperature anomaly - P4 (37°S, 50°W)", loc="left")
ax2.set_ylabel("SST (°C)")
### Final touch
fig.tight_layout()
plt.show()

#### 3. Determinação das constantes da Série de Fourier e reconstrução da série  
**a) Qual o comprimento de cada série (quantos anos)? Qual a frequência fundamental?**


In [ ]:
### ------------------------------------------------------------------------------------
### Return the info
### ------------------------------------------------------------------------------------
print(
    f"O comprimento da série de Chla é {len(chla_p4)} semanas (~{round(len(chla_p4)/46, 2)} anos) e sua frequencia fundamental é {round(1/len(chla_p4), 4)}"
)
print(
    f"O comprimento da série de SST é {len(sst_p4)} semanas (~{round(len(sst_p4)/46, 2)} anos) e sua frequencia fundamental é {round(1/len(sst_p4), 4)}"
)

**b) Calcule os coeficientes da Série de Fourier para ambas as séries (calculate_fourier_coef.m)**


In [ ]:
### ------------------------------------------------------------------------------------
### Define the function to calculate the coeff
### ------------------------------------------------------------------------------------
def calculate_fourier_coeff(series):
    series = series.reset_index(drop=True)
    N = len(series)
    p_array = np.arange(1, (N // 2) + 1, 1)
    n_array = np.arange(1, (N + 1), 1)
    a = []
    b = []
    T = []
    f = []
    for p in p_array:
        alpha = 0
        beta = 0
        for n in n_array:
            alpha = alpha + (2 / N) * (
                series.iloc[n - 1] * np.cos((2 * np.pi * n * p) / N)
            )
            beta = beta + (2 / N) * (
                series.iloc[n - 1] * np.sin((2 * np.pi * n * p) / N)
            )
        a.append(alpha)
        b.append(beta)
        T.append(N / p)
        f.append(p / N)

    A0 = 2 * series.mean()
    A = np.insert(a, 0, A0)
    B = np.insert(b, 0, 0)
    f = np.insert(f, 0, np.nan)
    T = np.insert(T, 0, np.nan)
    C = np.sqrt((A**2) + (B**2))
    theta = np.arctan2(B, A) * (180 / np.pi)
    theta[0] = np.nan
    data = {"A": A, "B": B, "C": C, "theta": theta, "f": f, "T": T}
    results_df = pd.DataFrame(data)
    return results_df

In [ ]:
### ------------------------------------------------------------------------------------
### Calculate the coeff
### ------------------------------------------------------------------------------------
coef_chla = calculate_fourier_coeff(chla_p4)
coef_sst = calculate_fourier_coeff(sst_p4)

**c) Monte uma pequena tabela, ordenando os coeficientes do maior para o menor, com as respectivas frequências e períodos**


In [ ]:
### ------------------------------------------------------------------------------------
### Make the table for chla
### ------------------------------------------------------------------------------------
coef_chla_sorted = coef_chla.sort_values(by=["C"], ascending=False).reset_index(
    drop=True
)
coef_chla_sorted.head(8)

In [ ]:
### ------------------------------------------------------------------------------------
### Make the table for sst
### ------------------------------------------------------------------------------------
coef_sst_sorted = coef_sst.sort_values(by=["C"], ascending=False).reset_index(drop=True)
coef_sst_sorted.head(8)

**d) Faça um gráfico de TSM x tempo, e outro de CHL x tempo, apenas para as frequências que apresentaram as maiores amplitudes (de 4 a 6 frequências, as dominantes). Utilize a função calculate_fourier_series2.m para a reconstrução das séries somente com as frequências obtidas**

In [ ]:
### ------------------------------------------------------------------------------------
### Define function to reconstruct the series
### ------------------------------------------------------------------------------------
def reconstruct_fourier_series(A, B, f, N):
    Na = len(A)
    Nb = len(B)
    if Na != Nb:
        print("Error: A and B must have the same size")
        return
    y = np.zeros((Na, N))
    i = np.arange(1, N + 1, 1)
    p_array = np.arange(Na)
    for p in p_array:
        y[p,] = A[p] * np.cos(2 * np.pi * f[p] * i) + B[p] * np.sin(
            2 * np.pi * f[p] * i
        )
    Y = 0.5 * A[0] + np.sum(y, axis=0)
    return y, Y

In [ ]:
### ------------------------------------------------------------------------------------
### Reconstruct the series (principal coeff)
### ------------------------------------------------------------------------------------
chla_p4_r_single, chla_p4_r_complete = reconstruct_fourier_series(
    coef_chla_sorted["A"][0:6],
    coef_chla_sorted["B"][0:6],
    coef_chla_sorted["f"][0:6],
    len(chla_p4),
)

sst_p4_r_single, sst_p4_r_complete = reconstruct_fourier_series(
    coef_sst_sorted["A"][0:6],
    coef_sst_sorted["B"][0:6],
    coef_sst_sorted["f"][0:6],
    len(sst_p4),
)

In [ ]:
### ------------------------------------------------------------------------------------
### Plot the principal coeff
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(10, 10), dpi=300)
ax1 = fig.add_subplot(411)
ax2 = fig.add_subplot(412, sharex=ax1)
ax3 = fig.add_subplot(413, sharex=ax1)
ax4 = fig.add_subplot(414, sharex=ax1)
### Subplot 1: Chla
chla_p4.plot(ax=ax1, color="green")
ax1.grid(linestyle="dashed", alpha=0.3)
ax1.set_title(
    "Chlorophyll-a Concentration anomaly - P4 (37°S, 50°W) | original data", loc="left"
)
ax1.set_ylabel("log$_{10}$[Chla] (mg $m^{-1}$)")
### Subplot 2: Chla-reconstructed
for n in np.arange(len(chla_p4_r_single)):
    ax2.plot(chla_p4.index, chla_p4_r_single[n], alpha=0.3, linestyle="dashed")
ax2.plot(chla_p4.index, chla_p4_r_complete, color="green")
ax2.grid(linestyle="dashed", alpha=0.3)
ax2.set_title(
    "Chlorophyll-a Concentration anomaly - P4 (37°S, 50°W) | reconstructed data (6 dominant f)",
    loc="left",
)
ax2.set_ylabel("log$_{10}$[Chla] (mg $m^{-1}$)")
### Subplot 3: SST
sst_p4.plot(ax=ax3, color="red")
ax3.grid(linestyle="dashed", alpha=0.3)
ax3.set_title(
    "Sea Surface Temperature anomaly - P4 (37°S, 50°W) | original data", loc="left"
)
ax3.set_ylabel("SST (°C)")
### Subplot 4: SST Reconstructed
for n in np.arange(len(sst_p4_r_single)):
    ax4.plot(sst_p4.index, sst_p4_r_single[n], alpha=0.3, linestyle="dashed")
ax4.plot(sst_p4.index, sst_p4_r_complete, color="red")
ax4.grid(linestyle="dashed", alpha=0.3)
ax4.set_title(
    "Sea Surface Temperature anomaly - P4 (37°S, 50°W) | reconstructed data (6 dominant f)",
    loc="left",
)
ax4.set_ylabel("SST (°C)")
### Final touch
fig.tight_layout()
plt.show()

**e) Reconstrua as séries (calculate_fourier_series2.m) com todas as frequências obtidas, desde a fundamental até a maior de todas**


In [ ]:
### ------------------------------------------------------------------------------------
### Reconstruct the series (all coeff)
### ------------------------------------------------------------------------------------
chla_p4_fullr_single, chla_p4_fullr_complete = reconstruct_fourier_series(
    coef_chla["A"][1:].reset_index(drop=True),
    coef_chla["B"][1:].reset_index(drop=True),
    coef_chla["f"][1:].reset_index(drop=True),
    len(chla_p4),
)

sst_p4_fullr_single, sst_p4_fullr_complete = reconstruct_fourier_series(
    coef_sst["A"][1:].reset_index(drop=True),
    coef_sst["B"][1:].reset_index(drop=True),
    coef_sst["f"][1:].reset_index(drop=True),
    len(sst_p4),
)

In [ ]:
### ------------------------------------------------------------------------------------
### Plot the reconstructed series
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(10, 5), dpi=300)
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212, sharex=ax1)
### Subplot 1: Chla
chla_p4.plot(ax=ax1, color="green", alpha=0.5, label="Original")
ax1.plot(
    chla_p4.index,
    chla_p4_fullr_complete,
    color="black",
    linestyle="dotted",
    alpha=0.6,
    label="Reconstructed",
)
ax1.grid(linestyle="dashed", alpha=0.3)
ax1.set_title("Chlorophyll-a Concentration anomaly - P4 (37°S, 50°W)", loc="left")
ax1.set_ylabel("log$_{10}$[Chla] (mg $m^{-1}$)")
ax1.legend()
### Subplot 2: SST
sst_p4.plot(ax=ax2, color="red", alpha=0.5, label="Original")
ax2.plot(
    sst_p4.index,
    sst_p4_fullr_complete,
    color="black",
    linestyle="dotted",
    alpha=0.6,
    label="Reconstructed",
)
ax2.grid(linestyle="dashed", alpha=0.3)
ax2.set_title("Sea Surface Temperature anomaly - P4 (37°S, 50°W)", loc="left")
ax2.set_ylabel("SST (°C)")
ax2.legend()
### Final touch
fig.tight_layout()
plt.show()

**f) Comente os resultados**  


Para as duas séries avaliadas, a frequência que apresentou a maior amplitude foi a frequência correspondente ao periodo de ~47 semanas, que representa o ciclo anual dos dados. Usando as primeiras 6 frequências para reconstruir as séries, foi possível obter uma reconstrução que representou muito bem as baixas frequências das duas séries avaliadas. Usando todos os coeficientes da série de Fourier, foi possível reconstruir integralmente ambas séries.

#### 4. Reconstrução da série apenas com a frequência anual
**a) Plote os ciclos anual e semi-anual de TSM e CHL sobrepostos às séries originais de TSM e CHL, com cores distintas**


In [ ]:
### ------------------------------------------------------------------------------------
### Calculate the semi-annual cycle of sst
### ------------------------------------------------------------------------------------
sst_semianual = reconstruct_fourier_series(
    coef_sst_sorted["A"][7:8].reset_index(drop=True),
    coef_sst_sorted["B"][7:8].reset_index(drop=True),
    coef_sst_sorted["f"][7:8].reset_index(drop=True),
    len(sst_p4),
)

In [ ]:
### ------------------------------------------------------------------------------------
### Plot the cycles
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(10, 5), dpi=300)
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212, sharex=ax1)
### Subplot 1: Chla
chla_p4.plot(ax=ax1, color="green", alpha=0.5, label="Original")
ax1.plot(
    chla_p4.index,
    chla_p4_r_single[2],
    color="blue",
    linestyle="dashed",
    alpha=0.8,
    label="Semi-Annual",
)
ax1.plot(chla_p4.index, chla_p4_r_single[0], color="black", alpha=0.8, label="Annual")
ax1.grid(linestyle="dashed", alpha=0.3)
ax1.set_title("Chlorophyll-a Concentration anomaly - P4 (37°S, 50°W)", loc="left")
ax1.set_ylabel("log$_{10}$[Chla] (mg $m^{-1}$)")
ax1.legend()
### Subplot 2: SST
sst_p4.plot(ax=ax2, color="red", alpha=0.5, label="Original")
ax2.plot(
    sst_p4.index,
    sst_semianual[0].flatten(),
    color="blue",
    linestyle="dashed",
    alpha=0.8,
    label="Semi-Annual",
)
ax2.plot(sst_p4.index, sst_p4_r_single[0], color="black", alpha=0.8, label="Annual")
ax2.grid(linestyle="dashed", alpha=0.3)
ax2.set_title("Sea Surface Temperature anomaly - P4 (37°S, 50°W)", loc="left")
ax2.set_ylabel("SST (°C)")
ax2.legend()
### Final touch
fig.tight_layout()
plt.show()

**b) Plote os resíduos entre as séries (original – ciclo anual). Ou seja, você está agora visualizando os dados de TSM e CHL sem o ciclo anual**

In [ ]:
### ------------------------------------------------------------------------------------
### Plot the residue
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(10, 5), dpi=300)
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212, sharex=ax1)
### Subplot 1: Chla
chla_p4.plot(ax=ax1, color="green", alpha=0.3, label="Original")
ax1.plot(
    chla_p4.index,
    chla_p4 - chla_p4_r_single[0],
    color="black",
    alpha=0.8,
    label="Residual",
)
ax1.grid(linestyle="dashed", alpha=0.3)
ax1.set_title("Chlorophyll-a Concentration anomaly - P4 (37°S, 50°W)", loc="left")
ax1.set_ylabel("log$_{10}$[Chla] (mg $m^{-1}$)")
ax1.legend()
### Subplot 2: SST
sst_p4.plot(ax=ax2, color="red", alpha=0.3, label="Original")
ax2.plot(
    sst_p4.index,
    sst_p4 - sst_p4_r_single[0],
    color="black",
    alpha=0.8,
    label="Residual",
)
ax2.grid(linestyle="dashed", alpha=0.3)
ax2.set_title("Sea Surface Temperature anomaly - P4 (37°S, 50°W)", loc="left")
ax2.set_ylabel("SST (°C)")
ax2.legend()
### Final touch
fig.tight_layout()
plt.show()

**c) Discuta os resultados**

Para as duas séries avaliadas, o ciclo anual foi construído utilizando somente os dados do coeficiente de Fourier correspondente a um período de 47 semanas. Dessa forma, a amplitude dessa frequência, isoladamente, não é significativa, tornando as séries residuais pouco distintas das séries originais. A variabilidade anual ainda é perceptível, sobretudo nos dados de temperatura. Uma abordagem que poderia aprimorar a eliminação dessa variabilidade anual seria empregar as seis frequências dominantes, que representam com maior precisão o ciclo anual de variação de temperatura e clorofila.